In [7]:
import requests
import json
import pandas as pd

In [1]:
print("hello")

hello


In [10]:
API_KEY = "I9FKdCn.hbfefNWCY336dL6x62vfwNKpoN2RZ1gp21"


In [9]:
import requests
import re
import pandas as pd

# API-Key for authentication
API_KEY = "I9FKdCn.hbfefNWCY336dL6x62vfwNKpoN2RZ1gp21"

# Base URL for fetching plenary protocols
base_url = 'https://search.dip.bundestag.de/api/v1/plenarprotokoll-text'

params = {
    'f.datum.start': '2023-09-27',  # Start date
    'f.datum.end': '2023-09-27',    # End date
    'apikey': API_KEY,
    'maxAnzahl': 1,
    'view': 'DETAIL'
}

# Request the plenary protocols
response = requests.get(base_url, params=params)

if response.status_code == 200:
    data = response.json()
    documents = data.get('documents', [])
    if documents:
        protokoll = documents[0]
        document_id = protokoll['id']
        print(f"Dokument-ID: {document_id}")
        
        # Zugriff auf den Text des Protokolls
        if 'text' in protokoll:
            text_data = protokoll['text']
            
            # Ausgabe eines größeren Ausschnitts des Textes
            print(f"Text des Protokolls (erste 2000 Zeichen):\n{text_data[:2000]}")
            
            # Angepasster regulärer Ausdruck
            pattern = r'\n([\w\s,.-]+?)\s*:\s*\n(.*?)(?=\n[\w\s,.-]+?\s*:\s*\n|\Z)'
            reden_matches = re.findall(pattern, text_data, re.DOTALL)
            print(f"Anzahl gefundener Reden: {len(reden_matches)}")
            
            reden_liste = []

            for match in reden_matches:
                redner_name = match[0].strip()
                rede_text = match[1].strip()
                reden_liste.append({
                    'redner_name': redner_name,
                    'rede_text': rede_text
                })

            # Überprüfen, ob die Redenliste nicht leer ist
            if reden_liste:
                # Erstellen des DataFrames
                df_reden = pd.DataFrame(reden_liste)
                print("Spalten in df_reden:", df_reden.columns.tolist())
                print(df_reden.head())
                
                # Aufteilen des Rednernamens in Vor- und Nachname
                df_reden[['vorname', 'nachname']] = df_reden['redner_name'].str.extract(r'^\s*(\S+)\s+(.*)$')

                # Anzeigen der ersten Zeilen mit aufgeteiltem Namen
                print(df_reden[['vorname', 'nachname', 'rede_text']].head())

                # Beispiel für eine einfache Analyse: Wortanzahl der Reden
                df_reden['wortanzahl'] = df_reden['rede_text'].apply(lambda x: len(x.split()))
                print("Durchschnittliche Wortanzahl der Reden:", df_reden['wortanzahl'].mean())

                # Optional: Speichern des DataFrames in einer CSV-Datei
                df_reden.to_csv('plenarprotokoll_reden.csv', index=False)
            else:
                print("Keine Reden gefunden. Bitte überprüfen Sie den regulären Ausdruck und den Text.")
        else:
            print("Der 'text'-Schlüssel ist nicht im 'protokoll' vorhanden.")
    else:
        print("Keine Dokumente gefunden.")
else:
    print(f"Fehler: {response.status_code}")

Dokument-ID: 5594
Text des Protokolls (erste 2000 Zeichen):
Plenarprotokoll 20/124
Deutscher Bundestag
Stenografischer Bericht
124. Sitzung
Berlin, Mittwoch, den 27. September 2023


Inhalt:

Erweiterung und Abwicklung der Tagesordnung 



Absetzung der Tagesordnungspunkte 21 und 28 b 



Tagesordnungspunkt 1:

Befragung der Bundesregierung 



Christian Lindner, Bundesminister BMF



Svenja Schulze, Bundesministerin BMZ



Dr. Mathias Middelberg (CDU/CSU)



Christian Lindner, Bundesminister BMF



Dr. Mathias Middelberg (CDU/CSU)



Christian Lindner, Bundesminister BMF



Katharina Beck (BÜNDNIS 90/DIE GRÜNEN)



Christian Lindner, Bundesminister BMF



Katharina Beck (BÜNDNIS 90/DIE GRÜNEN)



Christian Lindner, Bundesminister BMF



Peter Boehringer (AfD)



Christian Lindner, Bundesminister BMF



Peter Boehringer (AfD)



Christian Lindner, Bundesminister BMF



Dr. Karamba Diaby (SPD)



Svenja Schulze, Bundesministerin BMZ



Dr. Karamba Diaby (SPD)



Svenja Schulze, Bundesmi

In [17]:
import requests
import pandas as pd

# Base URL for the API
base_url = 'https://search.dip.bundestag.de/api/v1'

# Parameters for the initial API request to get the list of protocols
params = {
    'f.datum.start': '2023-09-27',  # Start date
    'f.datum.end': '2023-09-27',    # End date
    'apikey': API_KEY,
    'max': 1,                       # Limit the number of results
    'fulltext': 'false',            # Exclude full text for initial request
    'offset': 0                     # Start at the first result
}

# Step 1: Fetch the list of plenary protocols
response = requests.get(f"{base_url}/plenarprotokoll-text", params=params)
if response.status_code == 200:
    data = response.json()
    documents = data.get('documents', [])
    if documents:
        # Get the document ID of the first protocol
        protokoll = documents[0]
        document_id = protokoll['id']
        print(f"Dokument-ID: {document_id}")

        # Step 2: Fetch the detailed protocol using the document ID
        detail_url = f"{base_url}/plenarprotokoll/{document_id}"
        detail_params = {
            'apikey': API_KEY,
            'view': 'PLPR'  # Set view to 'PLPR' to get the 'text' key
        }
        detail_response = requests.get(detail_url, params=detail_params)

        if detail_response.status_code == 200:
            detail_data = detail_response.json()
            print("Strukturiertes Protokoll erfolgreich abgerufen.")

            # Check available keys
            print("Verfügbare Schlüssel im Protokoll:", detail_data.keys())

            # Access the 'text' key, which contains structured data
            if 'text' in detail_data:
                text_data = detail_data['text']
                # Check if 'reden' is in 'text'
                if 'reden' in text_data:
                    reden = text_data['reden']
                    reden_liste = []

                    # Step 4: Iterate over the speeches to extract information
                    for rede in reden:
                        redner_info = rede.get('redner', {})
                        rede_text_list = rede.get('p', [])
                        rede_text = ' '.join(rede_text_list)

                        # Extract speaker information
                        vorname = redner_info.get('vorname', '')
                        nachname = redner_info.get('nachname', '')
                        fraktion = redner_info.get('fraktion', '')
                        rolle = redner_info.get('rolle', '')

                        # Append the speech to the list
                        reden_liste.append({
                            'vorname': vorname,
                            'nachname': nachname,
                            'fraktion': fraktion,
                            'rolle': rolle,
                            'rede_text': rede_text
                        })

                    # Step 5: Create a DataFrame from the list of speeches
                    df_reden = pd.DataFrame(reden_liste)
                    print("Anzahl der extrahierten Reden:", len(df_reden))
                    print(df_reden.head())

                    # Step 6: Perform initial analysis
                    # Example: Count speeches per party
                    reden_pro_fraktion = df_reden['fraktion'].value_counts()
                    print("Anzahl der Reden pro Fraktion:")
                    print(reden_pro_fraktion)

                    # Example: Calculate average speech length
                    df_reden['wortanzahl'] = df_reden['rede_text'].apply(lambda x: len(x.split()))
                    durchschnitt_wortanzahl = df_reden['wortanzahl'].mean()
                    print(f"Durchschnittliche Wortanzahl pro Rede: {durchschnitt_wortanzahl:.2f}")

                    # Optional: Save the DataFrame to a CSV file
                    df_reden.to_csv('plenarprotokoll_reden.csv', index=False)
                else:
                    print("Der Schlüssel 'reden' ist nicht im 'text' vorhanden.")
            else:
                print("Der Schlüssel 'text' ist nicht im Protokoll vorhanden.")
        else:
            print(f"Fehler beim Abrufen des detaillierten Protokolls: {detail_response.status_code}")
    else:
        print("Keine Dokumente gefunden.")
else:
    print(f"Fehler bei der Anfrage: {response.status_code}")


Dokument-ID: 5594
Strukturiertes Protokoll erfolgreich abgerufen.
Verfügbare Schlüssel im Protokoll: dict_keys(['id', 'dokumentart', 'typ', 'vorgangsbezug_anzahl', 'dokumentnummer', 'wahlperiode', 'herausgeber', 'pdf_hash', 'xml_hash', 'aktualisiert', 'vorgangsbezug', 'fundstelle', 'titel', 'datum'])
Der Schlüssel 'text' ist nicht im Protokoll vorhanden.


In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

# Pfad zur heruntergeladenen XML-Datei
xml_file = 'path_to_your_downloaded_xml_file.xml'

# Parsing der XML-Datei
tree = ET.parse(xml_file)
root = tree.getroot()

# Angabe des Namespaces, falls vorhanden
ns = {'bt': 'http://www.bundestag.de/triton'}

reden_liste = []
for rede in root.findall('.//bt:rede', ns):
    # Extraktion der Rednerinformationen
    redner_elem = rede.find('bt:redner', ns)
    if redner_elem is not None:
        vorname = redner_elem.findtext('bt:vorname', default='', namespaces=ns)
        nachname = redner_elem.findtext('bt:nachname', default='', namespaces=ns)
        fraktion = redner_elem.findtext('bt:fraktion', default='', namespaces=ns)
    else:
        vorname = nachname = fraktion = ''

    # Extraktion des Redetextes
    rede_text_elems = rede.findall('bt:p', ns)
    rede_text = ' '.join([p.text for p in rede_text_elems if p.text])

    reden_liste.append({
        'vorname': vorname,
        'nachname': nachname,
        'fraktion': fraktion,
        'rede_text': rede_text
    })

# Erstellen eines DataFrames
df_reden = pd.DataFrame(reden_liste)
print("Anzahl der extrahierten Reden:", len(df_reden))
print(df_reden.head())